## Model simuleren: RTD model

    
$f:= (1 - e^{-(t-\tau_0)/\beta}), \forall t \geq \tau_0$ 
   

In [ ]:
%matplotlib inline

# Importeren van functionaliteiten
import numpy as np
import scipy as sp
import pandas as pd
import math
from IPython.display import Image
from ipywidgets import interact, fixed, FloatSlider, IntSlider, ToggleButtons

%load_ext autoreload
%autoreload 1
import sys
import os
sys.path.append("functions")
import general_functions

In [ ]:
%aimport rtd
model = rtd.expon_shifted

## Model simuleren

In [ ]:
df = rtd.calculate_rtd(
    np.arange(0, 100, 1), # tijdstappen
    model,
    returnDataFrame=True, plotresults=True,
    # vanaf hier: functie specifieke parameters
    beta=10, tau_0=30)

## Impact parameters

In [ ]:
interact(rtd.calculate_rtd,
         tijdstappen=fixed(np.arange(0, 100, 1)), # tijdstappen
         model=fixed(model),
         returnDataFrame=fixed(False), plotresults=fixed(True),
         # vanaf hier: functie specifieke parameters
         beta=FloatSlider(value=10, min=0.1, max=30, step=0.1, description='beta', continuous_update=False),
         tau_0=FloatSlider(value=30, min=5, max=80, step=1, description='tau_0', continuous_update=False),
         )

# Sensitiviteit

In [ ]:
interact(rtd.sensitiviteit,
    tijdstappen=fixed(np.arange(0, 100, 1)), # tijdstappen
    model=fixed(model), # initiële condities
    parameternaam=ToggleButtons(options=['beta', 'tau_0']),
    log_perturbatie = IntSlider(value=-4, min=-15, max=-1, step=1, continuous_update=False),
    soort=ToggleButtons(options=[
            'absolute sensitiviteit', 'relatieve sensitiviteit parameter',
            'relatieve sensitiviteit variabele','relatieve totale sensitiviteit']),
    # vanaf hier: functie specifieke parameters
    beta=fixed(20), tau_0=fixed(10))

# Calibratie

In [ ]:
df = pd.read_csv("./data/rtd_data_cpb_1_norm.csv", index_col=0)
time = df.index.values

In [ ]:
def sim_model(args):   
    beta, tau_0 = args
    sim = rtd.calculate_rtd(
        time, # tijdstappen
        model,
        returnDataFrame=True, plotresults=False,
        # vanaf hier: functie specifieke parameters
        beta=beta, tau_0=tau_0)
    return sim

def optimizer(args, obj_func=general_functions.sse, data=df):
    sim = sim_model(args)
    return obj_func(sim, data)

In [ ]:
x0 = [600, 600]
parameters, results = general_functions.track_calib(optimizer,
    x0, param_names=['beta', 'tau_0'], method='Nelder-Mead')
interact(rtd.plot_calib,
         parameters=fixed(parameters), results=fixed(results),
         i=IntSlider(value=0, min=0, max=parameters.shape[0]-1, step=1, continuous_update=False),
         data=fixed(df), sim_model=fixed(sim_model))
#general_functions.plot_calib(parameters, results, parameters.shape[0]-1, df_noisy)

In [ ]:
rtd.plot_contour_rtd(optimizer)